In [1]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
import os

In [3]:
def image_gen_w_aug(train_parent_directory, test_parent_directory):
    
    train_datagen = ImageDataGenerator(rescale=1/255,
                                      rotation_range = 30,  
                                      zoom_range = 0.2, 
                                      width_shift_range=0.1,  
                                      height_shift_range=0.1,
                                      validation_split = 0.15)
    
  
    
    test_datagen = ImageDataGenerator(rescale=1/255)
    
    train_generator = train_datagen.flow_from_directory(train_parent_directory,
                                                       target_size = (75,75),
                                                       batch_size = 214,
                                                       class_mode = 'categorical',
                                                       subset='training')
    
    val_generator = train_datagen.flow_from_directory(train_parent_directory,
                                                          target_size = (75,75),
                                                          batch_size = 37,
                                                          class_mode = 'categorical',
                                                          subset = 'validation')
    
    test_generator = test_datagen.flow_from_directory(test_parent_directory,
                                                     target_size=(75,75),
                                                     batch_size = 37,
                                                     class_mode = 'categorical')
    
    return train_generator, val_generator, test_generator


In [4]:
def model_output_for_TL (pre_trained_model, last_output):

    x = Flatten()(last_output)
    
    # Dense hidden layer
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.2)(x)
    
    # Output neuron. 
    x = Dense(3, activation='softmax')(x)
    
    model = Model(pre_trained_model.input, x)
    
    return model

In [5]:
train_dir = os.path.join('C:/Users/MayUMiiZ/Desktop/testweb/rps')
test_dir = os.path.join('C:/Users/MayUMiiZ/Desktop/testweb/rps-test-set/')

In [6]:
train_generator, validation_generator, test_generator = image_gen_w_aug(train_dir, test_dir)


Found 2142 images belonging to 3 classes.
Found 378 images belonging to 3 classes.
Found 372 images belonging to 3 classes.


In [7]:
pre_trained_model = InceptionV3(input_shape = (75, 75, 3), 
                                include_top = False, 
                                weights = 'imagenet')

for layer in pre_trained_model.layers:
  layer.trainable = False

In [8]:
last_layer = pre_trained_model.get_layer('mixed3')
last_output = last_layer.output

In [9]:
model_TL = model_output_for_TL(pre_trained_model, last_output)
model_TL.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [10]:
history_TL = model_TL.fit(
      train_generator,
      steps_per_epoch=10,  
      epochs=10,
      verbose=1,
      validation_data = validation_generator)


Epoch 1/10
10/10 [==============================] - 18s 2s/step - loss: 15.4695 - accuracy: 0.3859 - val_loss: 5.1086 - val_accuracy: 0.3333
Epoch 2/10
10/10 [==============================] - 9s 912ms/step - loss: 2.8390 - accuracy: 0.5176 - val_loss: 0.9819 - val_accuracy: 0.6852
Epoch 3/10
10/10 [==============================] - 10s 977ms/step - loss: 0.7714 - accuracy: 0.7469 - val_loss: 0.1582 - val_accuracy: 0.9392
Epoch 4/10
10/10 [==============================] - 10s 951ms/step - loss: 0.3247 - accuracy: 0.8833 - val_loss: 0.1855 - val_accuracy: 0.9365
Epoch 5/10
10/10 [==============================] - 10s 980ms/step - loss: 0.1306 - accuracy: 0.9477 - val_loss: 0.0731 - val_accuracy: 0.9762
Epoch 6/10
10/10 [==============================] - 10s 963ms/step - loss: 0.0913 - accuracy: 0.9673 - val_loss: 0.1546 - val_accuracy: 0.9471
Epoch 7/10
10/10 [==============================] - 11s 1s/step - loss: 0.0493 - accuracy: 0.9907 - val_loss: 0.0772 - val_accuracy: 0.9709
Epoch

In [11]:
tf.keras.models.save_model(model_TL,'my_model.hdf5')